<a id='top'></a>

# CSCI 3202, Spring 2018
# Assignment 5
# Due:  Wednesday 11 April 2018 by 12:00 PM

<br>

### Your name: Brennon Lee

<br>

**Note:** Some packages to load, helper functions and unit tests are defined at [the bottom of this notebook](#helpers). They're also defined up here, because I care.

Shortcuts:  [top](#top) || [1](#p1) | [1a](#p1a) | [1b](#p1b) | [1c](#p1c) | [1d](#p1d) | [1e](#p1e) | [1f](#p1f) | [1g](#p1g) || [2](#p2) | [2a](#p2a) | [2b](#p2b) | [2c](#p2c) | [2d](#p2d) | [2e](#p2e) || [3](#p3) | [3a](#p3a) | [3b](#p3b) | [3c](#p3c) | [3d](#p3d) | [3e](#p3e) | [3f](#p3f) | [3g](#p3g) || [helpers](#helpers)

In [51]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---

<a id='p1'></a>[Back to top](#top)

## Problem 1:  Bayesian network to model heart disease

The following Bayesian network is based loosely on a study that examined heart disease risk factors in 167 elderly individuals in South Carolina.  This study is [linked here](https://piazza.com/class_profile/get_resource/jc4v74a5uu5wa/jeyiv7kvs7r7ck) and posted to Piazza under the Resources tab.  Note that this figure uses Y and N to represent Yes and No, whereas in class we used the equivalent T and F to represent True and False Boolean values.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/hw05_bayesnet_heartdisease.png" style="width: 650px;"/>

<a id='p1a'></a>

### (1a) 

Create a `BayesNet` object to model this.  Below are the codes for the (conditional) probability `P` function and `BayesNode` class as well, that we used in class on Friday (16 March) to represent the variable nodes and calculate probabilities. You can code this however you want, subject to the following constraints:
1. the nodes are represented using the `BayesNode` class and can work with the `P` function for probabilities,
1. your `BayesNet` structure keeps track of which nodes are in the Bayes net, as well as
1. which nodes are the parents/children of which other nodes.

Some *suggested* skeleton codes for a class structure are given. The suggestions for methods to implement are in view of the fact that we will need to calculate some probabilities, which is going to require us to `find_node`s and `find_values` that nodes can take on.

In [12]:
## For the sake of brevity...
T, F = True, False

## From class:
def P(var, value, evidence={}):
    '''The probability distribution for P(var | evidence), 
    when all parent variables are known (in evidence)'''
    if len(var.parents)==1:
        # only one parent
        row = evidence[var.parents[0]]
    else:
        # multiple parents
        row = tuple(evidence[parent] for parent in var.parents)
    return var.cpt[row] if value else 1-var.cpt[row]

## Also from class:
class BayesNode:
    
    def __init__(self, name, parents, values, cpt):
        if isinstance(parents, str):
            parents = parents.split()
            
        if len(parents)==0:
            # if no parents, empty dict key for cpt
            cpt = {(): cpt}
        elif isinstance(cpt, dict):
            # if there is only one parent, only one tuple argument
            if cpt and isinstance(list(cpt.keys())[0], bool):
                cpt = {(v): p for v, p in cpt.items()}

        self.variable = name
        self.parents = parents
        self.cpt = cpt
        self.values = values
        self.children = []
        
    def __repr__(self):
        return repr((self.variable, ' '.join(self.parents)))    

    
##===============================================##
## Suggested skeleton codes for a BayesNet class ##
##===============================================##

class BayesNet:
    '''Bayesian network containing only boolean-variable nodes.'''

    def __init__(self, nodes):
        '''Initialize the Bayes net by adding each of the nodes,
        which should be a list BayesNode class objects ordered
        from parents to children (`top` to `bottom`, from causes
        to effects)'''
        
        # your code goes here...
        print(nodes)
        for node in nodes:
            self.add(node)
        

                
    def add(self, node):
        '''Add a new BayesNode to the BayesNet. The parents should all
        already be in the net, and the variable itself should not be'''
        assert node.variable not in self.variables
        assert all((parent in self.variables) for parent in node.parents)
        
        # your code goes here...
        
        

            
    def find_node(self, var):
        '''Find and return the BayesNode in the net with name `var`'''
        
        # your code goes here...
        

        
    def find_values(self, var):
        '''Return the set of possible values for variable `var`'''
        
        # your code goes here...
        

    
    def __repr__(self):
        return 'BayesNet({})'.format(self.nodes)

#### Unit tests

In [13]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem1("test_onenode"))
tests_to_run.addTest(Tests_Problem1("test_twonode"))
unittest.TextTestRunner().run(tests_to_run)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

<a/ id='p1b'></a>

### (1b)

Craft a function `get_prob(X, e, bn)` to return the **normalized** probability distribution of variable `X` in Bayes net `bn`, given the evidence `e`.  That is, return $P(X \mid e)$. The arguments are:
* `X` is some representation of the variable you are querying the probability distribution of. Either a string (the variable name from the `BayesNode` or a `BayesNode` object itself are good options.
* `e` is some representation of the evidence your probability is conditioned on. When given an empty argument (or `None`) for `e`, `get_prob` should return the marginal distribution $P(X)$.
* `bn` is your `BayesNet` object.

You may do this using the `enumeration` algorithm from class (pseudocode is in the book), or by brute force (i.e., use a few `for` loops). Either way, you should be using your `BayesNet` object to keep track of all the nodes and relationships between nodes so your `get_prob` function knows these things.

Use your `get_prob` function to calculate the following probabilities. Print them to the screen and compare to the original Bayes net figure given to make sure the output passes these "unit tests".

1. The marginal probability of `Family History` is $P(FH=T)=0.15$
2. The probability of *not* experiencing `Angina Pectoris`, given `Heart Disease` is observed, is $P(Ang=F \mid HD=T)=1-0.85=0.15$
3. The probability of `High Blood Pressure`, given a person does `Smoke and/or use Alcohol` but does not get `Moderate Exercise`, is $P(HBP=T \mid Sm=T, ME=F)=0.72$

<a/ id='p1c'></a>

### (1c)

Calculate the probability of observing someone with `High Blood Pressure`, $P(HBP=T)$, *by hand*, showing all work in Markdown/LateX below.

**Your answer:**

**Verify** your calculation using your `get_prob` function.

<a/ id='p1d'></a>

### (1d)

Now calculate the following probabilities using your `get_prob` function.

[i] The probability of an arbitrary individual having `Heart Disease`, $P(HD=T)$

[ii] The probability that an individual does *not* have `Heart Disease`, given that `Rapid Heartbeat` was observed, $P(HD=F \mid Rapid=T)$

[iii] The probability of an individual having `High Blood Pressure` if they have `Heart Disease` and a `Family History`, $P(HBP=T \mid HD=T, FH=T)$

[iv] The probability that an individual is a `Smoker/Alcohol User` if they have `Heart Disease`, $P(Sm=T \mid HD=T)$

[v] How would you expect the probability in [iv] to change if you also know the individual has `High Blood Pressure`?  Verify your hypothesis by calculating the relevant probability.

**Your answer:**

[vi] How would you expect the probability in [v] to change if you also know that the individual does *not* get `Moderate Exercise` (in addition to having `Heart Disease` and `High Blood Pressure`)?  Explain your answer using concepts from class.  Verify your answer by calculating the relevant probability.

**Your answer:**

---

<a id='p2'></a>[Back to top](#top)

<img src="https://inhabitat.com/wp-content/blogs.dir/1/files/2014/02/norman-bike-riding-dog.png" style="width: 350px;"/>

## Problem 2:  Bayesian network to model decision-making

Let's consider using a Bayesian network to model our decision about whether or not to ride our bike to work today.  This decision depends heavily on the weather, so let's focus on that.

In class, we focused on Boolean variables.  For example, we might base our biking decision on whether or not it is raining.  But in reality, it probably matters *how hard* it is raining.  So suppose we break the variable `Rain` up into three discrete bins: `none`, `light` and `heavy`.

The temperature also factors into our decision.  There is definitely a sweet spot, where temperatures are neither too warm nor too cold, so it is very likely we would enjoy riding our bike.  So we can model the variable `Temperature` also using three discrete bins: `cold`, `moderate` and `warm`.

So a Bayesian network to model our decision for whether or not to bike to work could be as follows, where the first letter of each discrete bin is used to denote that variable value (i.e., `R=h` stands for heavy rain conditions).

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/bayesnet_biking2.png" style="width: 650px;"/>

<a/ id='p2a'></a>

### (2a)

Modify the `P` probability function to be able to handle these ternary parent nodes.

Set up `BayesNode` objects for each of `Rain`, `Temp` and `Bike`, and create a `BayesNet` object to model the Bayesian network for this decision.  Again, you can use whatever structure you wish for your `BayesNet`, but please use the `BayesNode` class.  You may need to make minor modifications to the `BayesNode` class (e.g., changing/adding attributes), although none are strictly necessary.

**Verify** that your modified probability function `P` is working by checking the following "unit tests". Print the output to screen and compare to what you expect from the figure above.

1. The marginal probability of no rain is $P(Rain=n)=0.8$
1. The marginal probability of light rain is $P(Rain=l)=0.15$
1. The marginal probability of heavy rain is $P(Rain=h)=0.05$
1. The probability of biking given that it is raining heavily and the temperature is cold, is $P(Bike=T \mid Rain=h, Temp=c)=0.2$

<a/ id='p2b'></a>

### (2b)

Make any necessary modifications to your `get_prob` function from Problem 1, so that you can use it to calculate marginal probabilities and conditional probabilities for this problem. It is possible that your function does not require any modifications.

Use `get_prob` to calculate $P(Bike)$, the probability distribution for whether or not you will ride your bike on any given day.

Use `get_prob` to calculate the probability that you will ride your bike, given that it is lightly raining.

<a/ id='p2c'></a>

### (2c)

We are trapped indoors because some jerk gave us a ton of Intro to Artificial Intelligence homework to do.  Suppose we look out the window and see people biking. They sure do look like they're having fun! *Given* this information, we can actually make inferences regarding the temperature outside!  What is the probability distribution for temperature, given that we observe people biking?

First, compute this using your `get_prob` function.

<a/ id='p2d'></a>

### (2d)

Confirm your answer to **2c** by hand, showing *all* relevant work below in a LateX/Markdown cell.

**Your answer:**


<a/ id='p2e'></a>

### (2e)

Finally, confirm your confirmation of the probability distribution for `Temp` by using approximate Bayesian computation and 10,000 samples.  That is, use the **prior sampling** and **"rejection sampling"** techniques from class to estimate the probabilities associated with each possible value for `Temp`, given that there are people biking outside.

As a "Unit Test", check what the probability of riding your bike is, given no other information.  Make sure this approximately matches your answer to **2b**.

---

<a id='p3'></a>

## Problem 3:  Markov models - random walk to Taco Bell

Your friend Chris went to a party last night and drank way too much Fun Juice.  Chris is feeling a bit hungry, so he leaves the party, which is at the corner of 6th Street East and 3rd Street North, and heads for Taco Bell, which is located at the corner of 2nd Street East and 1st Street North.  A figure depicting this neighborhood is given below.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/random_walk_to_taco_bell.png" style="width: 650px;"/>

Fun Juice makes Chris' sense of direction quite poor, so at each intersection along his way, he picks any one of the available directions with equal probability.  Chris at least knows not to go north of 4th Street North, south of 0th Street North, east of 7th Street East, or west of 0th Street East, and has the common decency not to cut through anyone's yard (i.e., he only walks along streets).

Suppose Chris only cares about traveling between from one intersection to another, and considers one *move* to be walking one block, from one intersection to an adjacent intersection.

Since this grid is precisely a Cartesian coordinate grid, let the bottom-left corner of the neighborhood be represented by $(0,0)$.  Then the available moves from that location are to walk East (right, in the $+x$ direction) to $(1,0)$ or North to $(0,1)$.

<a id='p3a'></a>

### (3a)

Create a class for `Neighborhood` and for `Agent`, to represent the neighborhood and the agent trying to get to Taco Bell:

`Neighborhood(n_northsouth, n_eastwest, taco_bell)`:
* `n_northsouth` and `n_eastwest` provide the number of streets running north-south and the number of streets running east-west, respectively.  In the given figure, there are 5 streets running east-west, for example.
* `taco_bell` is a tuple providing the coordinates of Taco Bell in this neighborhood.
* has attributes for:
  * number of streets running north-south
  * number of streets running east-west
  * all of the intersections in the neighborhood
  * the location of the Taco Bell in the neighborhood
* Implement in your code a check to make sure the location of the Taco Bell is within the neighborhood's coordinates.  Assume that the south-west corner of the neighborhood is always $(0,0)$.

`Agent(name, loc, neighborhood)`:
* In the constructor, provide the agent with a `name` (string), initial location as a tuple (`loc`), and a `Neighborhood` to live in. Store these as attributes.
* Fill in the rest of the needed methods for the agent:
  * `available_moves()` returns a list of tuples, representing the locations the agent can walk to from its current location
  * `random_move()` returns one of the possible moves
  * `walk(move)` updates the agent's location, if they make the given argument `move`
  * `at_taco_bell()` returns True if the agent is at the same intersection as the neighborhood Taco Bell, and False otherwise

In [54]:
class Neighborhood:
    def __init__(self, n_northsouth, n_eastwest, taco_bell):
        '''Set up the layout of the neighborhood by giving the # streets
        running North/South (n_northsouth) and the # streets running 
        East/West). Based on these, store all the available locations 
        (intersections) in the neighborhood, and make sure the given
        coordinates for Taco Bell are valid'''
        
        # your code goes here...
        self.n_northsouth = n_northsouth
        self.n_eastwest = n_eastwest
        
        self.coords = []
        
        for ns in range(self.n_northsouth):
            for ew in range(self.n_eastwest):
                self.coords.append((ns, ew))
        
        assert(taco_bell in self.coords)
        
        self.taco_bell = taco_bell
        
        
class Agent:
    def __init__(self, name, loc, neighborhood):
        
        # your code goes here...
        self.name = name #string
        self.loc = loc #tuple
        self.neighborhood = neighborhood
        self.blocks = 0
        
    def available_moves(self):
        '''Return a list of available intersections the agent can move to,
        based on the layout of the agent's neighborhood'''
        moves = []
        
        right = (self.loc[0] + 1, self.loc[1])
        left = (self.loc[0] - 1, self.loc[1])
        up = (self.loc[0], self.loc[1] + 1)
        down = (self.loc[0], self.loc[1] - 1)
        
        check_moves = [right, left, up, down]
        
        for move in check_moves:
            if move in self.neighborhood.coords:
                moves.append(move)
        
        return moves

    def random_move(self):
        '''Return a random move out of the available moves
        from the agent`s current location'''
        moves = self.available_moves()
        idx = np.random.randint(0, len(moves))
    
        return moves[idx]
        
    
    def walk(self, move):
        '''Update the agent to a new location'''
        self.blocks += 1
        self.loc = move        
        
    def at_taco_bell(self):
        '''Return True if the agent is at Taco Bell, and False otherwise'''
        
        return self.loc == self.neighborhood.taco_bell
        
    
    def __repr__(self):
        return '{} at {}'.format(self.name, self.loc)

#### Unit tests

In [55]:
tests_to_run = unittest.TestSuite()
tests_to_run.addTest(Tests_Problem3('test_moves_corner'))
tests_to_run.addTest(Tests_Problem3('test_moves_center'))
tests_to_run.addTest(Tests_Problem3('test_tacos'))
unittest.TextTestRunner().run(tests_to_run)

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

<a/ id='p3b'></a>

### (3b)

Create a neighborhood and Chris agent to represent the situation above.

Then, run an ensemble of 1,000 simulations to obtain a sample for the number of blocks Chris must travel in order to arrive at Taco Bell at (2,1), starting from the party at (6,3).  Report the expected number of blocks traveled.

In [73]:
def chris_simulation():
    hood = Neighborhood(8,5,(2,1))
    Chris = Agent('Chris', (6,3), hood)

    while not Chris.at_taco_bell():
        move = Chris.random_move()
        Chris.walk(move)
    
    return Chris.blocks
        
num_of_blocks = [chris_simulation() for i in range(1000)]

print('Expected number of blocks traveled is: ', np.mean(num_of_blocks))

Expected number of blocks traveled is:  81.63


**Reflection:**  The sequence of states (coordinates) that Chris passed through in his travels is a **Markov chain** - each new state depended only on the previous one.  This process, called a **random walk**, can be a useful way to explore a state space.


<a/ id='p3c'></a>

### (3c)

Let us explore one of the characteristics of a Markov chain that we often find ourselves interested in:  the **expected time of first return** to a state.  In particular, let's examine Chris' **expected time of first return to Taco Bell**.

Build an ensemble of 1,000 simulations of how many blocks Chris must travel to return to Taco Bell, given that he starts at Taco Bell in the neighborhood depicted in the figure above.

We can estimate the expected time of first return to Taco Bell using the **mean** number of blocks Chris must travel in order to make it back to those tasty tacos. Report this estimate based on your simulation results.

In [72]:
def chris_first_return():
    hood = Neighborhood(8,5,(2,1))
    Chris = Agent('Chris', (2,1), hood)
    
    move = Chris.random_move()
    Chris.walk(move)
    
    while not Chris.at_taco_bell():
        move = Chris.random_move()
        Chris.walk(move)
    
    return Chris.blocks
        
num_of_blocks = [chris_first_return() for i in range(1000)]

print('Expected blocks traveled until first return is: ', np.mean(num_of_blocks))

Expected blocks traveled until first return is:  33.16


<a id='p3d'></a>

### (3d)

Your friend Dan also is leaving the party.  Dan leaves just after Chris.  Dan, however, has a preference for heading west (left, in the $-x$ direction) when he can, for that is where Ralphie roams.  In fact, his preference for being nearer to Ralphie is so strong that if it is possible for him to move west, then he does so 60% of the time, with the other 40% of the probability distributed equally among his other options.

Revise the `Agent` class - or better yet, *sub-class it*, to represent Dan's strange yet endearing decision-making for which random direction to move in.

In [58]:
class Dans_Agent:
    def __init__(self, name, loc, neighborhood):
        
        # your code goes here...
        self.name = name #string
        self.loc = loc #tuple
        self.neighborhood = neighborhood
        self.blocks = 0
        
    def available_moves(self):
        '''Return a list of available intersections the agent can move to,
        based on the layout of the agent's neighborhood'''
        moves = []
        
        right = (self.loc[0] + 1, self.loc[1]) #move one to the east
        left = (self.loc[0] - 1, self.loc[1]) #move one to the west
        up = (self.loc[0], self.loc[1] + 1) #move one up
        down = (self.loc[0], self.loc[1] - 1) #move one down
        
        check_moves = [right, left, up, down]
        
        for move in check_moves:
            if move in self.neighborhood.coords:
                moves.append(move)
        
        return moves

    def random_move(self):
        '''Return a random move out of the available moves
        from the agent`s current location'''
        moves = self.available_moves()
        
        west_move = (self.loc[0] - 1, self.loc[0])
        
        if west_move in moves:
            west_move_index = moves.index(west_move)
            probs = [0.4/(len(moves)-1) for i in range(len(moves))]
            probs[west_move_index] = 0.6
            idx = np.random.choice(len(moves), 1, p=probs)[0]
        else:
            idx = np.random.randint(0, len(moves))
    
        return moves[idx]
        
    
    def walk(self, move):
        '''Update the agent to a new location'''
        self.blocks += 1
        self.loc = move        
        
    def at_taco_bell(self):
        '''Return True if the agent is at Taco Bell, and False otherwise'''
        
        return self.loc == self.neighborhood.taco_bell
        
    
    def __repr__(self):
        return '{} at {}'.format(self.name, self.loc)

Now simulate 1,000 samples for how many blocks Dan will need to travel in order to make it to Taco Bell from the party.  Report the expected number of blocks Dan must travel to get some delicious chalupas.

In [93]:
def dans_simulation():
    dans_hood = Neighborhood(8,5,(2,1))
    Dan = Dans_Agent('Dan', (6,3), dans_hood)

    while not Dan.at_taco_bell():
        move = Dan.random_move()
        Dan.walk(move)
    
    return Dan.blocks
        
num_of_blocks = [dans_simulation() for i in range(1000)]

print('Expected number of blocks traveled for Dan is: ', np.mean(num_of_blocks))

Expected number of blocks traveled for Dan is:  84.744


<a/ id='p3e'></a>

### (3e)

What is Dan's expected time of first return to Taco Bell, as measured by the number of blocks traveled?

In [70]:
def dans_first_return():
    hood = Neighborhood(8,5,(2,1))
    Dan = Dans_Agent('Dan', (2,1), hood)
    
    move = Dan.random_move()
    Dan.walk(move)
    
    while not Dan.at_taco_bell():
        move = Dan.random_move()
        Dan.walk(move)
    
    return Dan.blocks
        
num_of_blocks = [dans_first_return() for i in range(1000)]

print('Expected blocks traveled until first return for Dan is: ', np.mean(num_of_blocks))

Expected blocks traveled until first return for Dan is:  41.338


### (3f)

Consider the smaller neighborhood depicted below, with only 3 north-south streets, 2 east-west streets, and a Taco Bell located at $(1,1)$.

<img src="http://www.cs.colorado.edu/~tonyewong/home/resources/random_walk_to_taco_bell_small.png" style="width: 250px;"/>

There are 6 distinct states in the state space, one for each of the 6 intersections.  Recall that the transition probability matrix denotes the probability of moving from the state given by the row of the matrix to the state given by the column.

Recall that the transition probability matrix is given by

$$T = \left[\begin{array}{cccccc} 
  q((0,0),(0,0)) & q((0,0),(1,0)) & q((0,0),(2,0)) & q((0,0),(0,1)) & q((0,0),(1,1)) & q((0,0),(2,1)) \\
  q((1,0),(0,0)) & q((1,0),(1,0)) & q((1,0),(2,0)) & q((1,0),(0,1)) & q((1,0),(1,1)) & q((1,0),(2,1)) \\
  q((2,0),(0,0)) & q((2,0),(1,0)) & q((2,0),(2,0)) & q((2,0),(0,1)) & q((2,0),(1,1)) & q((2,0),(2,1)) \\
  q((0,1),(0,0)) & q((0,1),(1,0)) & q((0,1),(2,0)) & q((0,1),(0,1)) & q((0,1),(1,1)) & q((0,1),(2,1)) \\
  q((1,1),(0,0)) & q((1,1),(1,0)) & q((1,1),(2,0)) & q((1,1),(0,1)) & q((1,1),(1,1)) & q((1,1),(2,1)) \\
  q((2,1),(0,0)) & q((2,1),(1,0)) & q((2,1),(2,0)) & q((2,1),(0,1)) & q((2,1),(1,1)) & q((2,1),(2,1)) \\  
\end{array} \right]$$

where $q(s_1, s_2)$ is the probability of moving from state $s_1$ to state $s_2$ in one step.

**For example:**  If Dan is currently at $(1,0)$, then:
* $q((1,0),(0,0))=0.6$
* $q((1,0),(2,0))=0.2$
* $q((1,0),(1,1))=0.2$
* $q((1,0),(0,1))=q((1,0),(2,1))=0$
* $q((1,0),(1,0))=0$ because Dan must move somewhere.

This row is already filled in for you.

Finish filling in the rest of the transition probability matrices below for each of Chris and Dan.

In [97]:
T_dan = np.array([[ 0 , 0.5,  0 , 0.5,  0 , 0  ],
                  [0.6,  0 , 0.2,  0 , 0.2, 0  ],
                  [ 0 , 0.6,  0 ,  0 ,  0 ,0.4 ],
                  [0.5,  0 ,  0 ,  0 , 0.5, 0  ],
                  [ 0 , 0.2,  0 , 0.6,  0 ,0.2 ],
                  [ 0 ,  0 , 0.4,  0 , 0.6, 0  ]])

T_chris = np.array([[ 0 ,0.5 , 0  , 0.5,  0 , 0  ],
                    [0.33, 0 ,0.33, 0  ,0.33, 0  ],
                    [ 0 , 0.5,  0 , 0  , 0 , 0.5 ],
                    [0.5,  0 ,  0 , 0  ,0.5,  0  ],
                    [ 0 ,0.33,  0 ,0.33, 0 ,0.33 ],
                    [ 0 ,  0 , 0.5, 0  ,0.5,  0  ]])

Use your transition probability matrices to estimate the probabilities (independently) that each of Chris and Dan will be at Taco Bell in 2 moves, given that they starts at $(0,0)$.

In [102]:
new_dan = T_dan
new_chris = T_chris
Dan_two_moves = np.matmul(new_dan, T_dan)
Chris_two_moves = np.matmul(new_chris, T_chris)
print('Probability that Dan will be at Taco Bell in 2 moves is', Dan_two_moves[0][4])
print('Probability that Chris will be at Taco Bell in 2 moves is', Chris_two_moves[0][4])

Probability that Dan will be at Taco Bell in 2 moves is 0.35
Probability that Chris will be at Taco Bell in 2 moves is 0.41500000000000004


What is the probability that either are at Taco Bell after 3 moves, if they start from $(0,0)$?  Why is this?

**Your answer:**

In [104]:
Dan_three_moves = np.matmul(Dan_two_moves, T_dan)
Chris_three_moves = np.matmul(Chris_two_moves, T_chris)
print('Probability that Dan will be at Taco Bell in 3 moves is', Dan_three_moves[0][4])
print('Probability that Chris will be at Taco Bell in 3 moves is', Chris_three_moves[0][4])

Probability that Dan will be at Taco Bell in 3 moves is 0.0
Probability that Chris will be at Taco Bell in 3 moves is 0.0


##### This is the case because starting at point $(0,0)$ and after making three moves, it's impossible to land on the point $(1,1)$

<a/ id='p3g'></a>

### (3g)

If Chris and Dan both spend all of their time walking around and eating at Taco Bell, what is the long-run proportion of Chris and Dan's time that each will spend at Taco Bell?  That is, if you run the random walk simulation for a very, very long time, what is your estimate for the proportion of states in the Markov chain that are at Taco Bell?

First, estimate these probabilities using long Markov chain random walk simulations for each of Chris and Dan. About 100,000 moves should be enough to stabilize your estimates.

In [119]:
hood = Neighborhood(3,2,(1,1))
Dan = Dans_Agent('Dan', (0,0), hood)
Chris = Agent('Chris', (0,0), hood)

Dan_taco_bell, Chris_taco_bell = 0,0
for i in range(100000):
    dan_move = Dan.random_move()
    chris_move = Chris.random_move()
    
    Dan.walk(dan_move)
    Chris.walk(chris_move)
    
    if Dan.at_taco_bell():
        Dan_taco_bell += 1
    if Chris.at_taco_bell():
        Chris_taco_bell += 1
    
print('Dan spends', Dan_taco_bell/100000, '% of his time at Taco Bell')
print('Chris spends', Chris_taco_bell/100000,'% of his time at Taco Bell')
    

Dan spends 0.21922 % of his time at Taco Bell
Chris spends 0.2151 % of his time at Taco Bell


Now estimate these probabilities using your transition probability matrices.  You should notice something a little bit strange when you compare these results against those from the simulations above.  Provide a couple sentences interpreting these results.

In [122]:
new_dan = T_dan
new_chris = T_chris
for k in range(1,100000):
    new_dan = np.matmul(new_dan, T_dan)
    new_chris = np.matmul(new_chris, T_chris)
    
print('Dan \n', new_dan)
print('\nChris \n', new_chris)

Dan 
 [[0.47368421 0.         0.13157895 0.         0.39473684 0.        ]
 [0.         0.39473684 0.         0.47368421 0.         0.13157895]
 [0.47368421 0.         0.13157895 0.         0.39473684 0.        ]
 [0.         0.39473684 0.         0.47368421 0.         0.13157895]
 [0.47368421 0.         0.13157895 0.         0.39473684 0.        ]
 [0.         0.39473684 0.         0.47368421 0.         0.13157895]]

Chris 
 [[5.99503073e-188 0.00000000e+000 5.99503073e-188 0.00000000e+000
  9.00545851e-188 0.00000000e+000]
 [0.00000000e+000 8.92820557e-188 0.00000000e+000 5.94360262e-188
  0.00000000e+000 5.94360262e-188]
 [5.99503073e-188 0.00000000e+000 5.99503073e-188 0.00000000e+000
  9.00545851e-188 0.00000000e+000]
 [0.00000000e+000 9.00545851e-188 0.00000000e+000 5.99503073e-188
  0.00000000e+000 5.99503073e-188]
 [5.94360262e-188 0.00000000e+000 5.94360262e-188 0.00000000e+000
  8.92820557e-188 0.00000000e+000]
 [0.00000000e+000 9.00545851e-188 0.00000000e+000 5.99503073e-188

**Your answer:**

<br><br><br>

<a id='helpers'></a>

---

[Back to top](#top)

## Some things that might be useful

Easiest way to start:  Click this cell, go to "Cell" in the toolbar above, and click "Run All Below"

In [52]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Unit tests

In [53]:
class Tests_Problem1(unittest.TestCase):
    def setUp(self):
        self.p1 = BayesNode('p1', '', [T,F], 0.3)
        self.p2 = BayesNode('p2', '', [T,F], 0.6)
        self.c  = BayesNode('c', ['p1', 'p2'], [T,F], {(T,T):0.1, (T,F):0.2, (F,T):0.3, (F,F):0.4})
    def test_onenode(self):
        self.assertEqual(P(self.p1, T), 0.3)
    def test_twonode(self):
        self.assertEqual(P(self.c, F, {'p1':T, 'p2':F}), 0.8)
        
class Tests_Problem3(unittest.TestCase):
    def test_moves_corner(self):
        nh = Neighborhood(2,2, (1,1))
        chris = Agent('Chris', (0,0), nh)
        self.assertTrue(((1,0) in chris.available_moves()) and ((0,1) in chris.available_moves()))
    def test_moves_center(self):
        nh = Neighborhood(3,3, (1,1))
        chris = Agent('Chris', (1,1), nh)
        self.assertTrue(((1,0) in chris.available_moves()) and ((0,1) in chris.available_moves()) and
                        ((1,2) in chris.available_moves()) and ((2,1) in chris.available_moves()))
    def test_tacos(self):
        nh = Neighborhood(3,3, (1,1))
        chris = Agent('Chris', (1,1), nh)
        self.assertTrue(chris.at_taco_bell())

[Back to top](#top)